In [31]:
import numpy as np
import pandas as pd

In [32]:
train_set = pd.read_csv('../../train.csv')
test_set = pd.read_csv('../../test.csv')

In [33]:
# checking NAN values
print(any(train_set.isnull().any()))
print(any(test_set.isnull().any()))

False
False


In [34]:
train_set.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

In [35]:
#replace 'String' with numeric values for target value

train_set['Species'] = train_set['Species'].map({'Iris-versicolor':1, 'Iris-virginica':2, 'Iris-setosa':3})
test_set['Species'] = test_set['Species'].map({'Iris-versicolor':1, 'Iris-virginica':2, 'Iris-setosa':3})

In [36]:
train_set

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,85,5.4,3.0,4.5,1.5,1
1,123,7.7,2.8,6.7,2.0,2
2,29,5.2,3.4,1.4,0.2,3
3,25,4.8,3.4,1.9,0.2,3
4,76,6.6,3.0,4.4,1.4,1
...,...,...,...,...,...,...
115,118,7.7,3.8,6.7,2.2,2
116,60,5.2,2.7,3.9,1.4,1
117,46,4.8,3.0,1.4,0.3,3
118,33,5.2,4.1,1.5,0.1,3
